In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=BblV6AQsd2s&ab_channel=RiseAgainstVEVO"

# Setting up the model
Define the LLM model to use

In [2]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

model.invoke("Are we in a simulation?")

AIMessage(content='There is currently no definitive evidence to prove or disprove the idea that we are living in a simulation. Some scientists and philosophers have put forward the theory that our reality could be a computer-generated simulation created by a more advanced civilization, similar to the concept portrayed in the movie "The Matrix." However, this is purely speculative and remains a topic of debate and exploration in the scientific community.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

# Using the magic of Chains :)

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# Chain example
chain = model | parser

chain.invoke("Are we in a simulation?")

'There is currently no definitive evidence to prove or disprove the idea that we are living in a simulation. It is a concept that has been explored by scientists, philosophers, and technologists, but ultimately remains a theoretical possibility. Without concrete evidence, it is up to individual interpretation and belief.'

# Using prompt templates

In [4]:
from langchain_core.prompts import ChatPromptTemplate

template = """
  Answer the question based on the context below. If you can't
  answer the question, just say "I don't know".

  Context: {context}

  Question: {question}
  """

prompt = ChatPromptTemplate.from_template(template)
print(prompt.format(context="The sky is blue.", question="What color is the sky?"))

print("Did it work?")

chain = prompt | model | parser
chain.invoke({
  "context": "I have an appointment tomorrow at 3pm.", 
  "question": "Can I go for lunch tomorrow at 3:00PM?"
})

Human: 
  Answer the question based on the context below. If you can't
  answer the question, just say "I don't know".

  Context: The sky is blue.

  Question: What color is the sky?
  
Did it work?


'No, you have an appointment at 3pm tomorrow.'

# Combining Chains

In [7]:
from operator import itemgetter

translation_template = """
  Translate the 

  answer: {answer}

  to

  language: {language}
  """

translation_prompt = ChatPromptTemplate.from_template(translation_template)

translation_chain = (
  {"answer": chain, "language": itemgetter("language") } | translation_prompt | model | parser
)

translation_chain.invoke({
  "context": "I have an appointment tomorrow at 3pm.", 
  "question": "Can I go for lunch tomorrow at 3:00PM?",
  "language": "es"
})


'Respuesta: No, tienes una cita a las 3pm mañana.'